In [1]:
import functools, itertools, timeit

q = 3329
assert is_prime(q)
psize = 256
vsize = 3
print("q =",q,"; psize =",psize, "; vsize =", vsize)

def qinv(x):
    """ multiplicative inverse in Fq """
    _, a, _ = xgcd(x,q)
    return a
    
def qsigned(x):
    """ signed view of 'x' """
    return x-q if q//2 < x else x

def norm_fracs(l, den):
    """ normalises franctions in 'l=[y1,...,yn]/den' """
    z = functools.reduce(math.gcd, l)
    z = math.gcd(z,den)
    return [ x//z for x in l], den//z



q = 3329 ; psize = 256 ; vsize = 3


In [2]:
class D:
    """ Simple distributions """
    def __init__(self, den, p0=None, l=None):
        """ constructor for simple distributions
            
            'den': common denominator
            Quasi-Uniform:
                p0 >= 0: (Pr[X=0] = p0/den)
                l = [y]: (Pr[X=k] = y/den, with k!=0)
            Symmetric:
                p0 < 0:
                l: half-sized list (Pr[X=k] = l[|k|]/den, with k!=0)
            General:
                p0 = None ()
                l: full-sized list (Pr[X=k] = l[k%q]/den)
        """
        if l == None:
            assert p0 != None and 0 <= p0, "wrong args for Distr"
        else:
            assert p0==None or p0<0, "wrong args for Distr"
        if p0 != None and 0 <= p0: # quasi-uniform
            [p0, p1], den = norm_fracs([p0*(q-1),den-p0],den*(q-1))
            self.p0 = p0
            self.l = [p1]
            self.den = den
        elif p0 != None and p0 < 0: # Symmetric
            self.p0 = -1
            self.den = den
            assert len(l)==(q+1)//2, "Wrong sized sym. distr. list"
            self.l = l
        else: # general
            self.p0 = None
            self.den = den
            assert len(l)==q, "Wrong sized distr. list"
            self.l = l
    def chk(self):
        """ checks if probs. sum to 1 """
        if self.p0 == None: # generic
            assert sum(self.l)/self.den == 1, " sum(generic distr) != 1 "
        elif self.p0 < 0: # symmetric
            assert (self.l[0]+2*sum(self.l[1:]))/self.den == 1, " sum(symmetric distr) != 1 "
        else: # quniform
            assert (self.p0 + (q-1)*self.l[0])/self.den == 1, " sum(quniform distr) != 1 "
    def dic(self):
        """ returns a PMF dictionary """
        m = {}
        for x in range(q):
            p = self.pr(x)
            if p!=0: m[x] = p
        return m
    def pr_num(self, x):
        """ numerator for the prob. of 'X=x' """
        if self.p0 == None: # general
            p = self.l[x]
        elif 0 <= self.p0: # quasi-uniform
            if x==0: p = self.p0
            else: p = self.l[0]
        elif self.p0 < 0: # symmetric
            if q//2 < x: p = self.l[q-x]
            else: p = self.l[x]
        return p
    def pr(self, x):
        """ prob. of 'X=x' """
        [p], d = norm_fracs([self.pr_num(x)],self.den)
        return p/d
    def pp(self):
        """ print PMF """
        for k in range((q+1)//2):
            print("Pr[%d]=" % k, float(self.pr(k)), "Pr[-%d]=" % k, float(self.pr((-k)%q)))
    def pr_ev(self, ev):
        """ prob. of an event 'ev' (a predicate) """
        p = 0
        for x in range(q):
            if ev(x): p += self.pr(x)
        return p
    def opp(self):
        """ computes (dmap self [-]) """
        if self.p0 != None: return self # quniform or symmetric
        # general case
        return D(self.den, None, [ self.pr_num((q-x)%q) for x in range(q) ])
    def full_l(self):
        """ returns the 'full' simple distribution """
        if self.p0 == None: # general
            return self.l
        if 0 <= self.p0: # quasi-uniform
            return [self.p0]+[self.l[0]*(q-1)]
        # symmetric
        return self.l + [ self.l[k] for k in range(q//2,0,-1) ]
    def sdist(self, d):
        """ computes the statistical distance between 'self' and 'd' """
        #assert self.p0 and d.p0 and 0<=self.p0 and 0<=d.p0, "'sdist' is only implemented on quniform dists."
        #dist = abs(self.pr(0)-d.pr(0)) + (q-1)*abs(self.pr(1)-d.pr(1))
        #return dist
        dist = 0
        for x in range(q):
            dist += abs(self.pr(x)-d.pr(x))
        return dist
    def conv(self, d):
        """ computes the convolution, i.e. 'dmap (self * d) (+)' """
        rden = self.den*d.den
        if self.p0 != None and 0 <= self.p0: # self is quniform
            f_x = lambda x: self.p0*x + self.l[0]*(d.den-x)
            if d.p0 != None and 0 <= d.p0: # d is quniform
                return D(rden, f_x(d.p0)) # result is quniform
            # d is symmetric or general
            return D(rden, d.p0, list(map(f_x, d.l)))
        if d.p0 != None and 0 <= d.p0: # d is quniform
            return d.conv(self)
        if self.p0 == None or d.p0 == None: # general convolution
            r = []
            for x in range(q):
                r.append(sum((self.pr_num(k)*d.pr_num((x-k)%q)
                             for k in range(q))))
            return D(rden, None, r)
        # both symmetric
        r = []
        for x in range((q+1)//2):
            r.append(sum((self.pr_num(k)*d.pr_num((x-k)%q)
                         for k in range(q))))
        return D(rden, -1, r)
    def n_conv(self, n):
        """ computes the n-fold convolution """
        r = D(1,1)
        n_bin = bin(n)[2:]  # binary representation of n
        for b in n_bin:
            r = r.conv(r)
            if b=='1': r = r.conv(self)
        return r
    def dmapmul(self, d):
        """ computes 'Pr[dmap (self * d) (*) | 0]' """
        if self.p0 != None and 0 <= self.p0: # self is quniform
            rden = self.den * d.den
            rp0 = d.den*self.p0 + self.den*d.pr_num(0) - self.p0*d.pr_num(0)
            return D(rden, rp0)
        if d.p0 != None and 0 <= d.p0: # d is quniform
            return d.dmapmul(self)
        rden = self.den * d.den
        rp0 = d.den*self.pr_num(0) + self.den*d.pr_num(0) - self.pr_num(0)*d.pr_num(0)
        if self.p0 != None and d.p0 != None: # both symmetric
            rl_tail = [ sum((2*self.pr_num(k)*d.pr_num((x*qinv(k))%q)
                             for k in range(1,(q+1)//2)))
                       for x in range(1,(q+1)//2) ]
            return D(rden,-1,[rp0]+rl_tail)
        # general case
        rl_tail = [ sum((self.pr_num(k)*d.pr_num((x*qinv(k))%q)
                        for k in range(1,q)))
                    for x in range(1,q) ]
        return D(rden, None, [rp0]+rl_tail)



# Polynomial multiplication

`dU` and `dN` are respectively the uniform and noise distributions on $\mathbb{F}_q$. We define also the distributions `dUNmul` and `dNNmul` resulting from combining them through multiplication on $\mathbb{F}_q$ (i.e., `dmap (dprod dU dN) (*)` and `dmap (dprod dN dN) (*)`).

In [3]:
# uniform distribution
dU = D(q,1)
dU.chk()

# noise distribution
dN = D(16, -1, [6,4,1] + [0]*((q+1)//2-3))
dN.chk()

# computed/explicit "(dmap (dprod dN dN) (*))
#dNNmul = dN.dmapmul(dN)
dNNmul = D(128, -1, [78,16,8,0,1] + [0]*((q+1)//2-5))
dNNmul.chk()

# computed/explicit "(dmap (dprod dU dN) (*))
#dUNmul = dU.dmapmul(dN)
dUNmul = D(26632, 9992)
dUNmul.chk()

We can now lift these distributions to polynomials on $\mathbb{F}_q[X]/(x^{256}+1)$. Specifically, we note that when $p_1$ and $p_2$ are polynomials whose coefficients are sampled according to `d1` and `d2` respectively, then each coefficient of $p_1 * p_2$ obbeys to the $256$-fold convolution of `d12mul`. Notice however that this observation only applies when we look at each coefficient independently, when `d1` and `d2` are symmetric (under the signed interpretation of $\mathbb{F}_q$).

In [4]:
dUNc = dUNmul.n_conv(psize)
#dUNc = D(5168332729269060045110952717091671006450173550981483018481904234588723621492948654595494222990564125097173655490749342703345189966858217109118032338612092012411630615173409335576833135867591222823300524315882124471722287176353656602624,
#         1552518092300708935148979488462502555256886017116696611139052038026050952686376886330878408828646477950488193317260584343842221763725250384261051395376671924175691036876553065599977148330410098713007875974205057973811793591137471744)
dUNc.chk()

dNNc = dNNmul.n_conv(psize)
dNNc.chk()
#dNNc.pp()

# Rounding Error
Part of the error expression if affected by a rounding error. We observe that what is affected by the rounding error is statistically close to the uniform distribution:
$$ A^T \cdot r + e_1 $$
where $A$ is a $3\times3$ matrix uniformly sampled, $r$ and $e_1$ are noise vectors.
Again, we are focusing on the distribution of a single coefficient, from a component of the vector.

In [5]:
# A * r
dAr = dUNc.n_conv(vsize)
dAr.chk()
# A * r + e1
dAre = dAr.conv(dN)
dAre.chk()

# statistical distance between 'dAre' and 'dU'
sdistU = dAre.sdist(dU)
boundU = ceil(log(sdistU)/log(2))
print("sdistU < 2^(%d):" % boundU, sdistU < 2^boundU)

# rounding error
def compress(x, rq=(2^10)):
    """ compress function """
    return round(1. * x * rq / q) % q

def decompress(x, rq=(2^10)):
    """ decompress function """
    return round(1. * x * q / rq)

def round_error (x):
    """ rounding error """
    return (x-decompress(compress(x)))

def test_round_error():
    for x in range(q):
        c = compress(x)
        d = decompress(c)
        e = round_error(x)
        print("comp/decomp(", x,")=", c, "->", d, "; e=", e)

def count_round_error():
    m = {}
    for x in range(q):
        r = round_error(x)
        m[r] = m.get(r,0) + 1
    return m

print("Error dist.:", count_round_error())

# round error distribution (over uniformly sampled coefficients)
dE = D(3329, None, [1024,1024, 128] + [0]*(q-5) + [129, 1024])
#dE.chk()
#dEopp = D(3329, None, [1024,1024, 129] + [0]*(q-5) + [128, 1024])
#assert (dE.opp().dic() == dEopp.dic())


sdistU < 2^(-1085): True
Error dist.: {0: 1024, 1: 1024, -1: 1024, -2: 129, 2: 128}


# Error Expression
The final error expression is given by:
$$ (e \cdot r) - (s \cdot (e_1-u)) + e_2 $$

EC source:
```python
 op noise_exp _A s e r e1 e2 m = 
    let t = _A *^ s + e in
    let u = m_transpose _A *^ r + e1 in
    let v = (t `<*>` r) +& e2 +& (m_encode m) in
    let cu = rnd_err_u u in
    let cv = rnd_err_v v in
          ((e `<*>` r) -&
           (s `<*>` e1) -&
           (s `<*>` cu) +& e2
          ) +& cv
```


In [6]:
#1: (e . r)[255]
d1 = dNNc.n_conv(vsize)
#2: (e1-cu)[0][255]
d2 = dN.conv(dE.opp())
#3: (s . (e1-cu))[255]
d3 = dN.dmapmul(d2).n_conv(psize*vsize)
#4: ((e . r) - (s . (e1-cu)))[255]
d4 = d1.conv(d3.opp())
#5: ((e . r) - (s . (e1-cu)) + e2)[255]
d5 = d4.conv(dN)

# Final failure probability
Failure probabily on a single coefficient is given by
$$ p = \mathrm{Pr}\left[ x \mid \left\lfloor \frac{q}{4} \right\rfloor - \textsf{cvbound} <= |x| \right] $$
where $\textsf{cvbound}$ is an upper bound on the absolute value of `cv`. Taking into account the statistical distance between `cu` and the rounding error under uniform dist. (`dE`), we get
$$ p' = p + \mathtt{sdistU}. $$
Hence, **final failure probability** is given by summing up the failure prob. for each coefficient, that is, $256 * p'$.

In [7]:
cvbound = 104
p = d5.pr_ev(lambda x: q//4-cvbound <= abs(qsigned(x)))
pfinal = psize*(p+sdistU)
boundP = ceil(log(pfinal)/log(2))
print("FailureProb = %d * (Pr[ x | q/4 - %d < |x| ] + 2^%d)" % (psize,cvbound,boundU))
print("FailureProb < 2^%d" % boundP)

FailureProb = 256 * (Pr[ x | q/4 - 104 < |x| ] + 2^-1085)
FailureProb < 2^-158
